In [ ]:
!pip install -q fairlearn

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from fairlearn.metrics import MetricFrame
from sklearn.metrics import accuracy_score,precision_score,recall_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
import seaborn as sns
from fairlearn.metrics import selection_rate
from fairlearn.reductions import ExponentiatedGradient, DemographicParity
from fairlearn.preprocessing import CorrelationRemover

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
pd.options.display.max_columns=500


from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder
label_encoder=LabelEncoder()

In [3]:
# data = fetch_openml(data_id=1590, as_frame=True)
#data = fetch_adult()
data = pd.read_csv("/kaggle/input/70k-job-applicants-data-human-resource/stackoverflow_full.csv").drop("Unnamed: 0",axis=1)
data["race"] = 0

import pandas as pd
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

def preprocess_data(df):
    """
    Preprocess a pandas DataFrame by:
    1. Filling missing values in numeric columns with median
    2. Filling missing values in categorical columns with mode
    3. Label encoding categorical columns
    4. MinMax scaling numeric columns to range [0,1]
    
    Parameters:
    df (pandas.DataFrame): Input DataFrame
    
    Returns:
    pandas.DataFrame: Preprocessed DataFrame
    """
    # Create a copy to avoid modifying the original DataFrame
    df = df.copy()
    
    # Identify numeric and categorical columns
    numeric_columns = df.select_dtypes(include=['float64', 'int64']).columns.drop("Employed")
    categorical_columns = df.select_dtypes(include=['object']).columns
    
    # Fill numeric columns with median
    df[numeric_columns] = df[numeric_columns].fillna(df[numeric_columns].median())
    
    # Fill categorical columns with mode
    df[categorical_columns] = df[categorical_columns].fillna(df[categorical_columns].mode().iloc[0])
    
    # Label encode categorical columns
    le = LabelEncoder()
    for col in categorical_columns:
        df[col] = le.fit_transform(df[col])
    
    # MinMax scale numeric columns
    scaler = MinMaxScaler()
    df[numeric_columns] = scaler.fit_transform(df[numeric_columns])
    df[categorical_columns] = scaler.fit_transform(df[categorical_columns])
    
    return df

# Example usage:
data = preprocess_data(data)
data = data.rename({"Gender":"sex","Employed":"target"},axis=1)

X = data
y = data.target
sex = data['sex']
race =data['race']
X_raw = data
A = X_raw["sex"]

df = pd.concat([X,pd.Series(y, name='>50K')], axis=1)

df.drop("target",axis=1,inplace=True)

In [4]:
df.head()

,Age,Accessibility,EdLevel,Employment,sex,MentalHealth,MainBranch,YearsCode,YearsCodePro,Country,PreviousSalary,HaveWorkedWith,ComputerSkills,race,>50K
0,0.0,0.0,0.00,1.0,0.0,0.0,0.0,0.14,0.08,0.853801,0.230139,0.541191,0.037383,0.0,0
1,0.0,0.0,1.00,1.0,0.0,0.0,0.0,0.24,0.10,0.824561,0.207505,0.246374,0.112150,0.0,1
2,0.0,0.0,0.00,1.0,0.0,0.0,0.0,0.30,0.12,0.315789,0.345042,0.585218,0.065421,0.0,0
3,0.0,0.0,1.00,1.0,0.0,0.0,0.0,0.18,0.12,0.157895,0.205956,0.253833,0.121495,0.0,0
4,1.0,0.0,0.75,0.0,0.0,0.0,1.0,0.80,0.60,0.789474,0.718445,0.539776,0.018692,0.0,0


In [ ]:
#correlation heatmap of dataset
def correlation_heatmap(df):
    _ , ax = plt.subplots(figsize =(14, 12))
    colormap = sns.diverging_palette(220, 10, as_cmap = True)
    
    _ = sns.heatmap(
        df.corr(), 
        cmap = "YlGn",
        square=True, 
        cbar_kws={'shrink':.9 }, 
        ax=ax,
        annot=True, 
        linewidths=0.1,vmax=1.0, linecolor='white',
        annot_kws={'fontsize':12 }
    )
    
    plt.title('Pearson Correlation of Features', y=1.05, size=15)

correlation_heatmap(df)

# Our Model

In [6]:
X=df.drop('>50K',axis=1)
y=df['>50K']

In [7]:
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
import numpy as np

y_pred = np.zeros(len(y))
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
classifier = CatBoostClassifier(verbose=0)

for train_idx, val_idx in skf.split(X, y):
    classifier.fit(X.loc[train_idx,:], y[train_idx])
    y_pred[val_idx] = classifier.predict(X.loc[val_idx,:])


## Fairness Assessment for the model 

# Metrics

In [8]:
metrics = {
    'accuracy': accuracy_score,
    'precision': precision_score,
    'recall': recall_score,
    'selection_rate': selection_rate
}

# -   Sex as Sensitive Feature 

In [ ]:
model_Assessmt_sex = MetricFrame(metrics=metrics, y_true=y, y_pred=y_pred, sensitive_features=df['sex'])
model_Assessmt_sex.by_group.plot.bar(
    subplots=True,
    layout=[3, 3],
    legend=False,
    figsize=[12, 8],
    title="Show all metrics",
    ylim=[0.6,1.0]
   
)
print(model_Assessmt_sex.overall)
print(model_Assessmt_sex.by_group)

# 2.   Democratic Parity Differences

## 2.1 Democratic Parity Differences for Sex

In [ ]:
from fairlearn.metrics import demographic_parity_difference
print(demographic_parity_difference(y_true=y,
                                    y_pred=y_pred,
                                    sensitive_features=df['sex']))
from fairlearn.metrics import demographic_parity_ratio
print(demographic_parity_ratio(y_true=y,
                               y_pred=y_pred,
                               sensitive_features=df['sex']))

#                               3. MITIGATION 

# 1)  Pre-processing

-    Executing Matigation Algorithms independently from each other.  

##   A- CorrelationRemover

 ###   a ) Sex as Sensitive Feature

In [11]:
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
import numpy as np

crp_sex = CorrelationRemover(sensitive_feature_ids=['sex'])
crp_sex.fit(X.fillna(0))
X_transform_sexp = crp_sex.transform(X.fillna(0))

y_pred_sexp = np.zeros(len(y))
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
classifier = CatBoostClassifier(verbose=0)

for train_idx, val_idx in skf.split(X_transform_sexp, y):
    classifier.fit(X_transform_sexp[train_idx], y[train_idx])
    y_pred_sexp[val_idx] = classifier.predict(X_transform_sexp[val_idx])


 ###   b ) Testing Fairness

In [ ]:
Metric_Corr_Para_sex = MetricFrame(metrics=metrics,
                           y_true=y,
                           y_pred=y_pred_sexp,
                           sensitive_features= df['sex'])
Metric_Corr_Para_sex.by_group.plot.bar(
    subplots=True,
    layout=[3, 3],
    legend=False,
    figsize=[12, 8],
    title="Show all metrics",
)
print(Metric_Corr_Para_sex.overall)

# B) Democratic Parity Differences 


 ###   a ) Sex as Sensitive Feature

In [ ]:
from fairlearn.metrics import demographic_parity_difference
print(demographic_parity_difference(y_true=y,
                                    y_pred=y_pred_sexp,
                                    sensitive_features=df['sex']))
from fairlearn.metrics import demographic_parity_ratio
print(demographic_parity_ratio(y_true=y,
                               y_pred=y_pred_sexp,
                               sensitive_features=df['sex']))

#  2 )  In-processing

## 2.1 ExponentiatedGradient

 ###   a ) Sex as Sensitive Feature

In [14]:
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
import numpy as np
np.random.seed(0)


y_pred_mitigatedP = np.zeros(len(y))
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for train_idx, val_idx in skf.split(X, y):
    constraint = DemographicParity()
    classifier = CatBoostClassifier(verbose=0)
    classifier = ExponentiatedGradient(classifier, constraint)
    classifier.fit(X.loc[train_idx,:], y[train_idx], sensitive_features=X.loc[train_idx,:][['sex']])
    y_pred_mitigatedP[val_idx] = classifier.predict(X.loc[val_idx,:])


In [ ]:
eg_sex_mitigated = MetricFrame(metrics=metrics, y_true=y, y_pred=y_pred_mitigatedP, sensitive_features=df[['sex']])


eg_sex_mitigated.by_group.plot.bar(
    subplots=True,
    layout=[3, 3],
    legend=False,
    figsize=[12, 8],
    title="Show all metrics",
)

In [ ]:
print(eg_sex_mitigated.overall)
print(eg_sex_mitigated.by_group)

## 2.2 Democratic Parity Differences 

 ###   a ) Sex as Sensitive Feature

In [ ]:
from fairlearn.metrics import demographic_parity_difference
print(demographic_parity_difference(y_true=y,
                                    y_pred=y_pred_mitigatedP,
                                    sensitive_features=df['sex']))
from fairlearn.metrics import demographic_parity_ratio
print(demographic_parity_ratio(y_true=y,
                               y_pred=y_pred_mitigatedP,
                               sensitive_features=df['sex']))

# 2.3 GridSearch 

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from fairlearn.reductions import DemographicParity, GridSearch, EqualizedOdds

 ###   a ) Sex as Sensitive Feature

In [21]:
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
import numpy as np
np.random.seed(0)


y_pred_GSp_sex_mitigated = np.zeros(len(y))
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for train_idx, val_idx in skf.split(X, y):
    constraint = DemographicParity()
    classifier = CatBoostClassifier(verbose=0)
    classifier = GridSearch(classifier, constraint,grid_size=31)
    classifier.fit(X.loc[train_idx,:], y[train_idx], sensitive_features=X.loc[train_idx,:][['sex']])
    y_pred_GSp_sex_mitigated[val_idx] = classifier.predict(X.loc[val_idx,:])


In [ ]:
GS_mitigatedp_sex = MetricFrame(metrics=metrics, y_true=y, y_pred=y_pred_GSp_sex_mitigated, sensitive_features=df[['sex']])
GS_mitigatedp_sex.by_group.plot.bar(
    subplots=True,
    layout=[3, 3],
    legend=False,
    figsize=[12, 8],
   title="Show all metrics",
)

In [ ]:
print(GS_mitigatedp_sex.overall)


## 2.4 Democratic Parity Differences 

 ###   a ) Sex as Sensitive Feature

In [ ]:
from fairlearn.metrics import demographic_parity_difference
print(demographic_parity_difference(y_true=y,
                                    y_pred=y_pred_GSp_sex_mitigated,
                                    sensitive_features=df['sex']))
from fairlearn.metrics import demographic_parity_ratio
print(demographic_parity_ratio(y_true=y,
                               y_pred=y_pred_GSp_sex_mitigated,
                               sensitive_features=df['sex']))

#  3 )  Post-processing

# 3.1 - ThresholdOptimizer

In [25]:
from fairlearn.postprocessing import ThresholdOptimizer

 ###  a ) sex as Sensitive Feature

In [26]:
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
import numpy as np
np.random.seed(0)


thp_sex_y_pred = np.zeros(len(y))
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for train_idx, val_idx in skf.split(X, y):
    classifier = CatBoostClassifier(verbose=0)
    classifier = ThresholdOptimizer(estimator=classifier, constraints='demographic_parity')
    classifier.fit(X.loc[train_idx,:], y[train_idx], sensitive_features=X.loc[train_idx,:][['sex']])
    thp_sex_y_pred[val_idx] = classifier.predict(X.loc[val_idx,:], sensitive_features=X.loc[val_idx,:][['sex']])


In [ ]:
metric_frame_thp_sex = MetricFrame(metrics=metrics,
                           y_true=y,
                           y_pred=thp_sex_y_pred,
                           sensitive_features=df[['sex']])
metric_frame_thp_sex.by_group.plot.bar(
    subplots=True,
    layout=[3, 3],
    legend=False,
    figsize=[12, 8],
    title="Show all metrics",
)

In [ ]:
print(metric_frame_thp_sex.overall)


## 3.2 Democratic Parity Differences 

 ###  a ) sex as Sensitive Feature

In [ ]:
from fairlearn.metrics import demographic_parity_difference
print(demographic_parity_difference(y_true=y,
                                    y_pred=thp_sex_y_pred,
                                    sensitive_features=df['sex']))
from fairlearn.metrics import demographic_parity_ratio
print(demographic_parity_ratio(y_true=y,
                               y_pred=thp_sex_y_pred,
                               sensitive_features=df['sex']))

# 3.2 Mitigation in Series 

# 1 ) CorrelationRemover + ExponentiateGradient

In [ ]:
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
import numpy as np
from tqdm import tqdm

crp_sex = CorrelationRemover(sensitive_feature_ids=['sex'])
crp_sex.fit(X.fillna(0))
X_transform_sexp = crp_sex.transform(X.fillna(0))

y_pred_mitigated_sex_s = np.zeros(len(y))
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
classifier = CatBoostClassifier(verbose=0)

for train_idx, val_idx in tqdm(skf.split(X_transform_sexp, y)):
    constraint = DemographicParity()
    classifier = CatBoostClassifier(verbose=0)
    classifier = ExponentiatedGradient(classifier, constraint,max_iter=10)
    classifier.fit(X_transform_sexp[train_idx], y[train_idx], sensitive_features=X.loc[train_idx,:][['sex']])
    y_pred_mitigated_sex_s[val_idx] = classifier.predict(X_transform_sexp[val_idx])


In [ ]:
dmpds = demographic_parity_difference(y_true=y,
                                    y_pred=y_pred_mitigated_sex_s,
                                    sensitive_features=df['sex'])
dmpds

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y,y_pred_mitigated_sex_s)

 # 2 ) CorrelationRemover + GridSearch

In [ ]:
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
import numpy as np
from tqdm import tqdm

crp_sex = CorrelationRemover(sensitive_feature_ids=['sex'])
crp_sex.fit(X.fillna(0))
X_transform_sexp = crp_sex.transform(X.fillna(0))

y_pred_GS_mitigated_sex_scg = np.zeros(len(y))
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
classifier = CatBoostClassifier(verbose=0)

for train_idx, val_idx in tqdm(skf.split(X_transform_sexp, y)):
    constraint = DemographicParity()
    classifier = CatBoostClassifier(verbose=0)
    classifier = GridSearch(classifier, constraint,grid_size=31*5)
    classifier.fit(X_transform_sexp[train_idx], y[train_idx], sensitive_features=X.loc[train_idx,:][['sex']])
    y_pred_GS_mitigated_sex_scg[val_idx] = classifier.predict(X_transform_sexp[val_idx])


In [ ]:
dmpds = demographic_parity_difference(y_true=y,
                                    y_pred=y_pred_GS_mitigated_sex_scg,
                                    sensitive_features=df['sex'])
dmpds


In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y,y_pred_GS_mitigated_sex_scg)

# 3 ) CorrelationRemover + ThresholdOptimizer

In [36]:
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
import numpy as np

crp_sex = CorrelationRemover(sensitive_feature_ids=['sex'])
crp_sex.fit(X.fillna(0))
X_transform_sexp = crp_sex.transform(X.fillna(0))

th_sex_y_pred_sct = np.zeros(len(y))
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
classifier = CatBoostClassifier(verbose=0)

for train_idx, val_idx in skf.split(X_transform_sexp, y):
    constraint = DemographicParity()
    classifier = CatBoostClassifier(verbose=0)
    classifier = ThresholdOptimizer(estimator=classifier, constraints='demographic_parity')
    classifier.fit(X.loc[train_idx,:], y[train_idx], sensitive_features=X.loc[train_idx,:][['sex']])
    th_sex_y_pred_sct[val_idx] = classifier.predict(X.loc[val_idx,:], sensitive_features=X.loc[val_idx,:][['sex']])


In [ ]:
from fairlearn.metrics import demographic_parity_difference
dmpds = demographic_parity_difference(y_true=y,
                                    y_pred=th_sex_y_pred_sct,
                                    sensitive_features=df['sex'])
dmpds

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y,th_sex_y_pred_sct)